In [1]:
import open3d as o3d
import pickle
import threading
import numpy as np
import sklearn
from ai import cs
from pyntcloud import PyntCloud
import rpointhop
import utils
import kitti_utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# lidar_to_cam = np.asarray([-1.857739385241e-03, -9.999659513510e-01, -8.039975204516e-03, -4.784029760483e-03, -6.481465826011e-03, 8.051860151134e-03, -9.999466081774e-01, -7.337429464231e-02, 9.999773098287e-01, -1.805528627661e-03, -6.496203536139e-03, -3.339968064433e-01])
# lidar_to_cam = lidar_to_cam.reshape(3,4)
# ones = np.zeros((1,4))
# ones[0,3] = 1
# lidar_to_cam = np.concatenate((lidar_to_cam, ones),axis=0)

# lidar_to_cam = np.ones((4, 4))

LOG_FOUT = open('stereoKITTI_ICP.txt', 'a')

In [3]:
def plot_consecutive_point_cloud(pc_s, pc_t):
    pcd_s = o3d.geometry.PointCloud()
    pcd_s.points = o3d.utility.Vector3dVector(pc_s[:, 0:3])
    pcd_s.paint_uniform_color([1, 0, 0])

    pcd_t = o3d.geometry.PointCloud()
    pcd_t.points = o3d.utility.Vector3dVector(pc_t[:, 0:3])
    pcd_t.paint_uniform_color([0, 1, 0])

    o3d.visualization.draw_geometries([pcd_s, pcd_t])
    
def plot_correspondences(X,Y):

    # X -> N x 3 numpy array of points
    # Y -> N x 3 numpy array of points

    points = np.concatenate((X,Y),axis=0)
    lines = []
    for i in range(X.shape[0]):
        # lines.append([X.shape[0]])
        lines.append([i, i+X.shape[0]])
    lines = np.asarray(lines)
    colors = [[173/255, 255/255, 47/255] for i in range(len(lines))]
    line_set = o3d.geometry.LineSet()  
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)

    X_pcd = o3d.geometry.PointCloud()
    X_pcd.points = o3d.utility.Vector3dVector(X)

    Y_pcd = o3d.geometry.PointCloud()
    Y_pcd.points = o3d.utility.Vector3dVector(Y)

    X_pcd.paint_uniform_color([113/255, 121/255, 126/255])
    Y_pcd.paint_uniform_color([196/255, 30/255, 58/255])

    o3d.visualization.draw_geometries([X_pcd,Y_pcd,line_set])

In [6]:
def get_transformation_result(pc, Rt):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc.reshape(-1, 3))
    pcd.transform(Rt)
    
    return np.asarray(pcd.points)

def get_transformation_ICP(data_x, data_y, frame):

#     data_x = data[0]
#     data_y = data[1]

    data_x_c = o3d.geometry.PointCloud()
    data_x_c.points = o3d.utility.Vector3dVector(data_x)

    data_y_c = o3d.geometry.PointCloud()
    data_y_c.points = o3d.utility.Vector3dVector(data_y)

    result = o3d.pipelines.registration.registration_icp(data_x_c, data_y_c, 0.1)

    Rt = result.transformation

    kitti_utils.log_matrix(frame, Rt, LOG_FOUT)

    return Rt

In [7]:
# with open('pointhop.pkl', 'rb') as f:
#     params = pickle.load(f, encoding='latin')
# T_total = np.eye(4)
# T_total = np.array([1.000000e+00, 1.197625e-11, 1.704638e-10, 1.665335e-16, 1.197625e-11, 1.000000e+00, 3.562503e-10, -1.110223e-16, 1.704638e-10, 3.562503e-10, 1.000000e+00, 2.220446e-16, 0, 0, 0, 1])
# T_total = T_total.reshape(4,4)
# kitti_utils.log_matrix(0, T_total, LOG_FOUT)

for i in range(200):
    path = '\\Workspace\\SceneFlow\\datasets\\stereo_kitti\\'+str(i).zfill(6)+'.npz'
    pc_data = np.load(path)
#     data_x = pc_data['pos1']
#     data_y = pc_data['pos2']
    data_x = pc_data['pc1']
    data_y = pc_data['pc2']
    
#     attribute_0, data_0 = kitti_utils.attribute(data_x)
#     attribute_1, data_1 = kitti_utils.attribute(data_y)
#     attribute_h = np.concatenate((attribute_0, attribute_1), axis=0)
#     data = np.concatenate((data_0, data_1), axis=0)

#     leaf_node = rpointhop.pointhop_pred(data, attribute_h, pca_params=params, n_sample=[36,8,16])
#     features = np.moveaxis(np.squeeze(np.asarray(leaf_node)), 0, 2)
    Rt = get_transformation_ICP(data_x, data_y, i)
    
#     data_x_w = get_transformation_result(data_x, Rt)
#     plot_consecutive_point_cloud(data_x_w, data_y)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
